# Unified Experiments Runner (Colab-ready)

This notebook sets up the environment (Colab or local) and runs all experiment combinations across datasets and architectures.

- Datasets: BraTS, MSD Liver, TotalSegmentator
- Architectures: UNet, UNETR, SegResNet

If running on Colab, it mounts Google Drive and installs requirements. Use the runner cell to execute all combinations or adjust selections.


In [ ]:
# Detect Colab and optionally mount Google Drive
IN_COLAB = False
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

print(f"Running in Colab: {IN_COLAB}")

if IN_COLAB:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')

# Ensure correct working directory when cloned in Colab
import os
import sys
from pathlib import Path
import subprocess

if IN_COLAB:
    # Support current repo name (typo) and future corrected name
    candidates = [
        (
            Path('/content/3d_medical_segemntation'),
            'https://github.com/thabhelo/3d_medical_segemntation.git'
        ),
        (
            Path('/content/3d_medical_segmentation'),
            'https://github.com/thabhelo/3d_medical_segmentation.git'
        ),
    ]

    # Use existing clone if present; otherwise try cloning candidates in order
    repo_dir = None
    for path, url in candidates:
        if path.exists():
            repo_dir = path
            break
    if repo_dir is None:
        for path, url in candidates:
            try:
                subprocess.run(['git','clone','-q', url, str(path)], check=True)
                repo_dir = path
                break
            except Exception:
                continue
    if repo_dir is None:
        raise RuntimeError('Failed to clone repository using known names.')

    os.chdir(repo_dir)

sys.path.insert(0, str(Path.cwd() / 'src'))
print(f"CWD: {Path.cwd()}")


In [ ]:
# Optionally install dependencies (Colab)
if IN_COLAB:
    import subprocess
    # Install minimal pinned deps; setup.sh is heavy, so install selectively for speed
    cmds = [
        ['pip','install','-q','torch==2.3.0','torchvision==0.18.0','--index-url','https://download.pytorch.org/whl/cu121'],
        ['pip','install','-q','monai[all]==1.3.0','numpy==1.26.4','scipy>=1.12,<1.14'],
        ['pip','install','-q','scikit-learn>=1.3.0','pandas>=2.0.0'],
        ['pip','install','-q','matplotlib>=3.7.0','seaborn>=0.12.0','nibabel','SimpleITK','PyYAML','tqdm','tensorboard']
    ]
    for c in cmds:
        print('Running:', ' '.join(c))
        subprocess.run(c, check=True)
    print('Dependencies installed.')
else:
    print('Local environment detected; ensure requirements are installed.')


In [ ]:
# Runner: iterate over datasets x architectures
from pathlib import Path
import itertools
import subprocess
import sys

# Configurable selections
DATASETS = ['brats', 'msd_liver', 'totalsegmentator']
ARCHITECTURES = ['unet', 'unetr', 'segresnet']
MAX_EPOCHS = 2
BATCH_SIZE = 2
NUM_WORKERS = 2
OUTPUT_BASE = Path('results/colab_runs')

OUTPUT_BASE.mkdir(parents=True, exist_ok=True)

print('Dataset root auto-detected by train_model.py based on environment.')

runs = list(itertools.product(DATASETS, ARCHITECTURES))
print(f'Total runs: {len(runs)}')

for ds, arch in runs:
    out_dir = OUTPUT_BASE / f'{ds}_{arch}'
    cmd = [
        sys.executable,
        'scripts/train_model.py',
        '--dataset', ds,
        '--architecture', arch,
        '--max_epochs', str(MAX_EPOCHS),
        '--batch_size', str(BATCH_SIZE),
        '--num_workers', str(NUM_WORKERS),
        '--output_dir', str(out_dir)
    ]
    print('\n=== Running:', ' '.join(cmd))
    subprocess.run(cmd, check=False)

print('\nAll runs completed.')


# SegResNet Training Experiments\n
\n
This notebook trains the remaining SegResNet experiments:\n
1. SegResNet + BraTS\n
2. SegResNet + MSD Liver\n
\n
**Prerequisites:**\n
- Run `00_environment_setup.ipynb` first\n
- Ensure GPU is available and datasets are uploaded\n
\n
**Expected Duration:** ~45 minutes total

In [ ]:
# Quick verification that environment is ready\n
import torch\n
import sys\n
import os\n
from pathlib import Path\n
from datetime import datetime\n
import subprocess\n
\n
print('Pre-training verification:')\n
print('-' * 30)\n
\n
# Check GPU\n
if torch.cuda.is_available():\n
    print(f'GPU: {torch.cuda.get_device_name(0)}')\n
else:\n
    print('WARNING: No GPU detected - training will be slow')\n
\n
# Determine dataset path\n
is_colab = 'google.colab' in sys.modules or os.path.exists('/content')\n
datasets_root = Path('/content/drive/MyDrive/datasets' if is_colab else Path.home() / 'Downloads/datasets')\n
\n
# Check required datasets\n
required_datasets = ['BraTS', 'MSD']\n
for dataset in required_datasets:\n
    dataset_path = datasets_root / dataset\n
    print(f'{dataset}: {\"Available\" if dataset_path.exists() else \"MISSING\"}')\n
\n
print('\nReady for SegResNet experiments!')

## SegResNet + BraTS Training

In [ ]:
print(f'Starting SegResNet + BraTS at {datetime.now().strftime(\"%H:%M:%S\")}')\n
print('Expected duration: ~20 minutes')\n
print('=' * 50)\n
\n
subprocess.run([\n
    'python', 'scripts/train_model.py',\n
    '--dataset', 'brats',\n
    '--data_root', str(datasets_root),\n
    '--architecture', 'segresnet',\n
    '--in_channels', '4',\n
    '--out_channels', '4',\n
    '--batch_size', '2',\n
    '--max_epochs', '100',\n
    '--output_dir', 'results/segresnet_brats'\n
], check=True)\n
\n
print('SegResNet + BraTS completed!')

## SegResNet + MSD Liver Training

In [ ]:
print(f'Starting SegResNet + MSD Liver at {datetime.now().strftime(\"%H:%M:%S\")}')\n
print('Expected duration: ~25 minutes')\n
print('=' * 50)\n
\n
subprocess.run([\n
    'python', 'scripts/train_model.py',\n
    '--dataset', 'msd_liver',\n
    '--data_root', str(datasets_root),\n
    '--architecture', 'segresnet',\n
    '--in_channels', '1',\n
    '--out_channels', '3',\n
    '--batch_size', '2',\n
    '--max_epochs', '100',\n
    '--output_dir', 'results/segresnet_msd_liver'\n
], check=True)\n
\n
print('SegResNet + MSD Liver completed!')

## Training Summary

In [ ]:
# Check training results\n
results_dir = Path('results')\n
segresnet_experiments = ['segresnet_brats', 'segresnet_msd_liver']\n
\n
print('SegResNet Training Summary')\n
print('=' * 40)\n
\n
completed_count = 0\n
for exp in segresnet_experiments:\n
    exp_dir = results_dir / exp\n
    best_model = exp_dir / 'best.pth'\n
    \n
    if best_model.exists():\n
        completed_count += 1\n
        model_size = best_model.stat().st_size / (1024 * 1024)\n
        print(f'COMPLETED: {exp} ({model_size:.1f} MB)')\n
    else:\n
        print(f'NOT FOUND: {exp}')\n
\n
print(f'\nProgress: {completed_count}/{len(segresnet_experiments)} SegResNet experiments completed')\n
print('Next: Run 02_train_totalsegmentator_experiments.ipynb')